# Master Metrics for L2 Retrospecive Grants

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ramshreyas/retrospective_grants_metrics/blob/main/master_metrics.ipynb)

---

### Setup & Imports

Installs

In [ ]:
!pip install pandas

Imports

In [ ]:
from google.colab import files
import pandas as pd
import io
from datetime import datetime
import requests
import json
from IPython.display import display, JSON, clear_output
from pprint import pprint
from time import sleep
import warnings
warnings.filterwarnings('ignore')

Repo owner and name extraction function

In [ ]:
def extract_github_owner_repo(df, url_column):
    results = []
    for url in df[url_column]:
        # Check if the URL is a string and contains the expected 'github.com'
        if isinstance(url, str) and 'github.com/' in url:
            # Split the URL to isolate the owner and repo
            parts = url.split('github.com/')[-1].split('/')
            if len(parts) >= 2:
                owner, repo = parts[0], parts[1]
                results.append((owner, repo))
            else:
                # Handle cases where URL might not correctly split into owner/repo
                results.append((parts[0], None))
        else:
            # Append None or placeholders if URL is not valid or missing
            results.append((None, None))
    return results

GraphQL Query Function

In [ ]:
def execute_graphql_query(query, variables=None, token="your_github_token_here"):
    """
    Executes a GraphQL query to the GitHub API.

    Args:
    query (str): The GraphQL query to execute.
    variables (dict, optional): Variables required for the query. Defaults to None.
    token (str): GitHub API token for authentication.

    Returns:
    dict: The JSON response from the API.
    """
    # Endpoint for GitHub's GraphQL API
    url = 'https://api.github.com/graphql'

    # Headers including the authentication token
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }

    # Package the request
    payload = json.dumps({
        'query': query,
        'variables': variables
    })

    # Make the HTTP POST request
    response = requests.post(url, headers=headers, data=payload)

    # Check if the response was successful
    if response.status_code == 200:
        response_data = response.json()  # Parse JSON response
        display(JSON(response_data))  # Display the response in JSON format for clarity
        return response_data
    else:
        print(f"Query failed with status code {response.status_code}: {response.text}")
        return None

HTTP Query Function

In [ ]:
def execute_http_query(url, method='GET', headers=None, data=None, params=None, token=None):
    """
    Executes a generic HTTP request to the GitHub API.

    Args:
    url (str): The full URL to the GitHub API endpoint.
    method (stra): The HTTP method to use ('GET', 'POST', etc.).
    headers (dict, optional): Additional HTTP headers as a dictionary.
    data (dict, optional): Data to send with the request (for POST, PUT methods).
    params (dict, optional): URL parameters to append to the request.
    token (str, optional): GitHub API token for authentication.

    Returns:
    tuple: (response content as JSON if available, headers of the response)
    """
    # Initialize headers if not provided
    if headers is None:
        headers = {}
    # Add the authorization token to headers if provided
    if token:
        headers['Authorization'] = f'token {token}'
    headers.setdefault('Accept', 'application/vnd.github.v3+json')

    try:
        # Make the HTTP request
        response = requests.request(method, url, headers=headers, json=data, params=params)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        return response.json(), response.headers
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e.response.status_code} {e.response.reason}")
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError:  # Includes simplejson.decoder.JSONDecodeError
        return response.text, response.headers  # Return plain text if JSON decoding fails

Pull Requests

In [ ]:
def fetch_total_pull_requests(owner, repo, token):
    """
    Fetches the total number of pull requests for a given repository.

    Args:
    owner (str): GitHub repository owner.
    repo (str): GitHub repository name.
    token (str): GitHub API token.

    Returns:
    int: Total number of pull requests.
    """
    base_url = f'https://api.github.com/repos/{owner}/{repo}/pulls'
    params = {'per_page': 100, 'state': 'all'}  # Fetch all PRs, both open and closed
    total_pull_requests = 0

    while True:
        print(f"Fetching URL: {base_url} with params: {params}")  # Debugging output
        response, headers = execute_http_query(base_url, params=params, token=token)
        total_pull_requests += len(response)
        print(f"Found {len(response)} pull requests on this page. Total so far: {total_pull_requests}")  # Debugging output

        if 'next' in headers.get('Link', ''):
            links = headers['Link'].split(',')
            next_link = [link for link in links if 'rel="next"' in link]
            if next_link:
                next_url = next_link[0].split(';')[0].strip(' <>')
                base_url = next_url  # Update the URL to the next page's URL
                print(f"Next page URL: {next_url}")  # Debugging output
            else:
                break
        else:
            break

    return total_pull_requests

Issues

In [ ]:
def get_issues(owner, repo, token):
  query = """
  query GetRepositoryInfo($owner: String!, $repo: String!) {
    repository(owner: $owner, name: $repo) {
      all:issues {
        totalCount
      }
      closed:issues(states:CLOSED) {
        totalCount
      }
      open:issues(states:OPEN) {
        totalCount
      }
    }
  }
  """

  variables = {
      "owner": owner,
      "repo": repo
  }

  # Execute the query
  response = execute_graphql_query(query, variables, token)

  all = response["data"]["repository"]["all"]["totalCount"]
  closed = response["data"]["repository"]["closed"]["totalCount"]
  open = response["data"]["repository"]["open"]["totalCount"]

  return all, closed, open

Issue-Resolution turnaround time

In [ ]:
def calculate_average_turnaround(owner, repo, token):
    # Query
    query = """
    query GetIssueTurnaroundTimes($owner: String!, $repo: String!) {
      repository(owner: $owner, name: $repo) {
        issues(first: 100) {  # Adjust 'first' for pagination
          edges {
            node {
              createdAt
              closedAt
            }
          }
          pageInfo {
            endCursor
            hasNextPage
          }
        }
      }
    }
    """

    variables = {
          "owner": owner,
          "repo": repo
      }

    # Execute the query
    data = execute_graphql_query(query, variables, token)

    times = []
    for edge in data['data']['repository']['issues']['edges']:
        if edge['node']['closedAt']:
            start = datetime.fromisoformat(edge['node']['createdAt'].replace('Z', '+00:00'))
            end = datetime.fromisoformat(edge['node']['closedAt'].replace('Z', '+00:00'))
            delta = end - start
            times.append(delta.total_seconds())

    average_seconds = sum(times) / len(times) if times else 0

    return average_seconds / 86400  # Convert seconds to days

---

### Load Grantee Sheet

In [ ]:
# Create and display the file upload widget
uploaded_files = files.upload()

Check if upload is ok

In [ ]:
filename, content = next(iter(uploaded_files.items()))
df = pd.read_csv(io.BytesIO(content))
metrics_df = df[['Opportunity Name', 'Account Name', 'Github', 'Contract: Completed Date', 'Contract: Contract Start Date', 'Contract: Contract End Date']]
metrics_df.head()

---

### Generate Metrics

In [ ]:
token = ""

In [ ]:
owner_repo_list = extract_github_owner_repo(metrics_df, 'Github')

In [ ]:
def generate_metrics(owner_repo_list):
  generic_data = []
  stars = []
  forks = []
  pulls = []
  issues = []
  issue_tots = []
  licenses = []

  for owner, repo in owner_repo_list:
    if repo:
      print(f"Fetching generic data for {owner}/{repo}")
      gd = execute_http_query(f'https://api.github.com/repos/{owner}/{repo}', token=token)[0]
      generic_data.append(gd)
      stars.append(gd["stargazers_count"])
      forks.append(gd["forks_count"])
      licenses.append(gd["license"])
      sleep(0.1)
      print(f"Fetching pull data for {owner}/{repo}")
      pulls.append(fetch_total_pull_requests(owner, repo, token))
      sleep(0.1)
      print(f"Fetching issue data for {owner}/{repo}")
      issues.append(get_issues(owner, repo, token))
      sleep(0.1)
      print(f"Calculating issue turnaround time for {owner}/{repo}")
      issue_tots.append(calculate_average_turnaround(owner, repo, token))
      sleep(0.1)
    else:
      print(f"No repo listed for {owner}")
      generic_data.append(None)
      stars.append(None)
      forks.append(None)
      pulls.append(None)
      issues.append(None)
      issue_tots.append(None)
      licenses.append(None)

    clear_output(wait=True)

  return generic_data, stars, forks, pulls, issues, issue_tots, licenses

In [ ]:
generic_data, stars, forks, pulls, issues, issue_tots, licenses = generate_metrics(owner_repo_list)

In [ ]:
metrics_df["stars"] = stars
metrics_df["forks"] = forks
metrics_df["pulls"] = pulls
metrics_df["issues"] = issues
metrics_df["issue_tots"] = issue_tots
metrics_df["license"] = licenses

In [ ]:
metrics_df

Donwload CSV

In [ ]:
# Save the CSV content to a file
with open('metrics_data.csv', 'w') as f:
    f.write(metrics_df.to_csv(index=False))

# Use the files.download() function to offer the file for download
files.download('metrics_data.csv')